In [ ]:
from molass_legacy import get_version
from molass_data import SAMPLE4
assert get_version(toml_only=True) >= "0.3.4", "Please update molass_legacy to version 0.3.4 or higher."
from molass_legacy._MOLASS.SerialSettings import clear_temporary_settings
from molass_legacy.Batch.StandardProcedure import StandardProcedure

clear_temporary_settings()
sp = StandardProcedure()
sd = sp.load_old_way(SAMPLE4)

In [ ]:
from importlib import reload
import molass_legacy.Optimizer.Scripting
reload(molass_legacy.Optimizer.Scripting)
from molass_legacy.Optimizer.Scripting import prepare_optimizer
optimizer, init_params = prepare_optimizer(SAMPLE4, sd=sd, num_components=3)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(ncols=3, figsize=(15, 4))
ax1, ax2, ax3 = axes
axt = ax2.twinx()
axt.grid(False)
axis_info = (fig, (*axes, axt))       # i.e., draw all
ax1.set_title("UV Decomposition", fontsize=16)
ax2.set_title("Xray Decomposition", fontsize=16)

optimizer.objective_func(init_params)
optimizer.objective_func(init_params, plot=True, axis_info=axis_info)

In [ ]:
from importlib import reload
import molass_legacy.Optimizer.Scripting
reload(molass_legacy.Optimizer.Scripting)
from molass_legacy.Optimizer.Scripting import run_optimizer
run_optimizer(SAMPLE4, optimizer, init_params, debug=True)

In [ ]:
import threading
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Simulated optimization process (runs in background)
progress = []
def optimization_process():
    for i in range(100):
        time.sleep(0.1)  # Simulate work
        progress.append(np.sin(i/10) + np.random.randn()*0.1)  # Simulate metric

# Start the background process
thread = threading.Thread(target=optimization_process)
thread.start()

# Monitor and plot progress
plt.ion()
fig, ax = plt.subplots()
line, = ax.plot([], [])
ax.set_ylim(-2, 2)
ax.set_xlim(0, 100)
ax.set_xlabel("Iteration")
ax.set_ylabel("Metric")

while thread.is_alive() or len(progress) < 100:
    line.set_data(range(len(progress)), progress)
    ax.set_xlim(0, max(100, len(progress)))
    clear_output(wait=True)
    display(fig)
    plt.pause(0.1)

# Final plot
line.set_data(range(len(progress)), progress)
clear_output(wait=True)
display(fig)
plt.ioff()

In [ ]:
import threading
import time
import matplotlib.pyplot as plt

progress = []

def watch_progress_file(filename):
    while True:
        try:
            with open(filename) as f:
                # Read and parse progress (e.g., as lines or JSON)
                progress[:] = [float(line.strip()) for line in f if line.strip()]
        except Exception as e:
            pass
        time.sleep(0.5)  # Adjust interval as needed

# Start the watcher thread
thread = threading.Thread(target=watch_progress_file, args=("progress.txt",), daemon=True)
thread.start()

# Plotting loop
plt.ion()
fig, ax = plt.subplots()
line, = ax.plot([], [])
ax.set_ylim(-2, 2)
ax.set_xlim(0, 100)

while True:
    y = list(progress)
    if y:
        line.set_data(range(len(y)), y)
        ax.set_xlim(0, max(100, len(y)))
    fig.canvas.draw_idle()
    plt.pause(0.5)
    # Add a break condition as needed

In [ ]:
import threading
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

progress_file = "progress.txt"

# Simulated optimization process: writes progress to a file
def optimization_process():
    with open(progress_file, "w") as f:
        for i in range(100):
            time.sleep(0.1)
            value = np.sin(i/10) + np.random.randn()*0.1
            f.write(f"{value}\n")
            f.flush()

# File-watching thread: reads progress from the file
progress = []
def watch_progress_file():
    last_len = 0
    while True:
        try:
            with open(progress_file) as f:
                lines = f.readlines()
                # Only update if new lines are added
                if len(lines) > last_len:
                    progress[:] = [float(line.strip()) for line in lines if line.strip()]
                    last_len = len(lines)
        except Exception:
            pass
        if last_len >= 100:
            break
        time.sleep(0.1)

# Start the optimization process in a thread
opt_thread = threading.Thread(target=optimization_process)
opt_thread.start()

# Start the file-watching thread
watch_thread = threading.Thread(target=watch_progress_file)
watch_thread.start()

# Plotting loop (main thread)
plt.ion()
fig, ax = plt.subplots()
line, = ax.plot([], [])
ax.set_ylim(-2, 2)
ax.set_xlim(0, 100)
ax.set_xlabel("Iteration")
ax.set_ylabel("Metric")

while watch_thread.is_alive() or len(progress) < 100:
    y = list(progress)
    if y:
        line.set_data(range(len(y)), y)
        ax.set_xlim(0, max(100, len(y)))
    clear_output(wait=True)
    display(fig)
    plt.pause(0.1)

# Final plot
y = list(progress)
if y:
    line.set_data(range(len(y)), y)
    ax.set_xlim(0, max(100, len(y)))
clear_output(wait=True)
display(fig)
plt.ioff()

In [ ]:
import multiprocessing as mp
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import os

progress_file = "progress.txt"

# Optimization process: writes progress to a file (runs in a subprocess)
def optimization_process(progress_file):
    with open(progress_file, "w") as f:
        for i in range(100):
            time.sleep(0.1)
            value = np.sin(i/10) + np.random.randn()*0.1
            f.write(f"{value}\n")
            f.flush()

# File-watching thread: reads progress from the file
progress = []
def watch_progress_file():
    last_len = 0
    while True:
        try:
            with open(progress_file) as f:
                lines = f.readlines()
                if len(lines) > last_len:
                    progress[:] = [float(line.strip()) for line in lines if line.strip()]
                    last_len = len(lines)
        except Exception:
            pass
        if last_len >= 100:
            break
        time.sleep(0.1)

if __name__ == "__main__":
    # Clean up old progress file if exists
    if os.path.exists(progress_file):
        os.remove(progress_file)

    mp.set_start_method("spawn", force=True)
    proc = mp.Process(target=optimization_process, args=(progress_file,))
    proc.start()

    # Start the file-watching thread
    watch_thread = threading.Thread(target=watch_progress_file)
    watch_thread.start()

    # Plotting loop (main thread)
    plt.ion()
    fig, ax = plt.subplots()
    line, = ax.plot([], [])
    ax.set_ylim(-2, 2)
    ax.set_xlim(0, 100)
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Metric")

    while watch_thread.is_alive() or len(progress) < 100:
        y = list(progress)
        if y:
            line.set_data(range(len(y)), y)
            ax.set_xlim(0, max(100, len(y)))
        clear_output(wait=True)
        display(fig)
        plt.pause(0.1)

    # Final plot
    y = list(progress)
    if y:
        line.set_data(range(len(y)), y)
        ax.set_xlim(0, max(100, len(y)))
    clear_output(wait=True)
    display(fig)
    plt.ioff()